In [1]:
import sys
sys.path.append('/eos/home-i00/l/lkoller/SWAN_projects/commands/Lya_commands')

from Ltif import Lpicday, Lpic
from sklearn import datasets, metrics, svm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from joblib import dump, load

In [2]:
target1 = np.array([1,1,1,1,1,1,1,2,1,2,1,1,2,1,1, #1st file
                  1,1,1,2,1,1,1,1,1,1,1,1,1,1, #2nd file
                  1,1,1,1,2,1,1,1,1,1, #3rd file
                  1,1,1,1,2,1,1,2,2,1,1,2,1, #4th file
                  1,1,2,1,1,1,1,2,2,1,2,1, #5th file !!!!! 1,1,2,1,1,1,1,2,2,1,5,1,
                  1,1,1,1,1,2,1,1,1,1,1, #6th file
                  1,1,1,1,1,2,1,1,1, #7th file !!!!!(the 2 might be a 1)
                  2,1,1,1,1,1,1,1,1,1,1,1, #8th file
                  1,1,1,1,1,2,1,2,1, #9th file
                  1,1,2,1,1,1,1,1, #10th file
                  1,1,1,1,1,1,1,1,2,1,1,2,2,1,1,1,1,1, #11th file
                  1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1, #12th file
                  1,2,1,1,1,1,1,1,1,1,1,1,2,1,1,1, #13th file
                  1,1,1,1,2,1,1,1,1,1,1,1,1,1,1, #1,1,1,1,1,2,1,1,1,1,1,1,1, #14th file
                  1,1,1,2,1,2,2,1,1,1,1,1,1,1,2, #15th file !!!!! 1,1,1,2,1,2,4,1,1,1,1,1,1,1,2,
                  1,1,1,1,1,1,1,1,1,1,1,1,2,1, #16th file
                  1,1,1,1,1,1,1,1,1,2,2,1,1,1,1,1,2,2,1,1,2,1,1, #17th file
                  1,1,2,1,1,1,1,1,1,1,2,1,1, #18th file
                  1,1,1,1,1,1,1,1,1,2,1, #19th file
                  1,1,1,1,1,1,1]) #20th file

target2 = np.array([1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,2,1,1,
                  2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,2,1,1,1,1,1,1,1,1,
                  1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  2,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,
                  1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,2,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                  1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1])

target = np.append(target1,target2)

In [3]:
df1 = Lpicday('23_12_14').clusters_day(min_size = 100)
df2 = Lpicday('23_12_06').clusters_day(min_size = 80)

100%|██████████| 474/474 [00:22<00:00, 21.07it/s]


In [4]:
cl1 = [val for bal in list(df1['cluster']) for val in bal if len(val) > 0]
cl2 = [val for bal in list(df2['cluster']) for val in bal if len(val) > 0]

cl = cl1 + cl2

max_h, max_w = 53, 42

for item in [val for bal in cl for val in bal]:
    while len(item) < max_w:
        item += [0]

for item in cl:
    while len(item) < max_h:
        item += [[0] * max_w]
        
ac = np.array(cl)

data = ac.reshape(len(ac), -1)

In [5]:
clf = svm.SVC()
clf.fit(data,target)
dump(clf,'/eos/user/l/lkoller/ML models/clust_num_v3.txt')

['/eos/user/l/lkoller/ML models/clust_num_v3.txt']